In [1]:
import pandas as pd
import numpy as np
import evaluation

In [2]:


path = '/Users/joro/src/lynx/data/close_prices_insample.csv'
prices = pd.read_csv(path, index_col='AsOfDate', parse_dates=['AsOfDate'])
ret = prices.diff()

In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load the data and parse the date column
data = pd.read_csv("data.csv", parse_dates=["AsOfDate"])
data.sort_values("AsOfDate", inplace=True)
data.set_index("AsOfDate", inplace=True)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/joro/src/lynx/.venv/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <54A1AE05-1E14-3DA2-A8D0-062134694298> /Users/joro/src/lynx/.venv/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no 

In [7]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load the data and parse the date column
data = pd.read_csv("data.csv", parse_dates=["AsOfDate"])
data.sort_values("AsOfDate", inplace=True)
data.set_index("AsOfDate", inplace=True)

# Function to create lag features
def create_lag_features(df, column, lags):
    for lag in lags:
        df[f"{column}_lag{lag}"] = df[column].shift(lag)
    return df

# Function to build and evaluate a model for a single target
def build_model_for_target(data, target, test_size=10, lags=[1, 2, 3], plot=True):
    # Create lag features and a rolling mean feature
    df = data.copy()
    df = create_lag_features(df, target, lags=lags)
    df[f"{target}_rolling3"] = df[target].rolling(window=3).mean()
    
    # Drop rows with missing values from lag/rolling operations
    df.dropna(inplace=True)
    
    # Split the data into training and test sets
    train = df.iloc[:-test_size]
    test = df.iloc[-test_size:]
    
    # Define features (all lag and rolling features) and labels (the target)
    features = [col for col in df.columns if (f"{target}_lag" in col or f"{target}_rolling" in col)]
    X_train = train[features]
    y_train = train[target]
    X_test = test[features]
    y_test = test[target]
    
    # Build and train the LightGBM model
    model = LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)
    
    # Make predictions and evaluate the model
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    print(f"LightGBM Mean Squared Error for {target}:", mse)
    
    if plot:
        # Plot actual vs predicted values
        plt.figure(figsize=(10, 5))
        plt.plot(test.index, y_test, label="Actual")
        plt.plot(test.index, preds, label="Predicted")
        plt.legend()
        plt.title(f"LightGBM Forecast vs Actual for {target}")
        plt.xlabel("Date")
        plt.ylabel(target)
        plt.show()
    
    return model, features

# List of target variables to predict
targets = ["ganymede_stocks"]  # Add all your target columns here

# Dictionary to store models and their features
models = {}

# Build a model for each target
for target in targets:
    if target in data.columns:
        print(f"\nBuilding model for {target}")
        model, features = build_model_for_target(data, target)
        models[target] = {"model": model, "features": features}
    else:
        print(f"Warning: {target} not found in the dataset")

# Example of how to use a model to make predictions on new data
def predict_with_model(new_data, target, model_dict):
    model = model_dict[target]["model"]
    features = model_dict[target]["features"]
    
    # Ensure all required features are present in the new data
    for feature in features:
        if feature not in new_data.columns:
            print(f"Warning: {feature} not found in new data. Creating it...")
            # Extract the base column name and lag/rolling info
            if "_lag" in feature:
                col = feature.split("_lag")[0]
                lag = int(feature.split("_lag")[1])
                new_data[feature] = new_data[col].shift(lag)
            elif "_rolling" in feature:
                col = feature.split("_rolling")[0]
                window = int(feature.split("_rolling")[1])
                new_data[feature] = new_data[col].rolling(window=window).mean()
    
    # Make predictions
    return model.predict(new_data[features])

OSError: dlopen(/Users/joro/src/lynx/.venv/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /Users/joro/src/lynx/.venv/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file)

Python version: 3.11.6
Python architecture: ('64bit', '')
System architecture: arm64
